# செமாந்திக் கர்னல் கருவி பயன்பாட்டு உதாரணம்


## தேவையான தொகுப்புகளை இறக்குமதி செய்யவும்


In [1]:
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## பிளகின்களை உருவாக்குதல்    
Semantic Kernel பிளகின்களை ஏஜென்ட் அழைக்கக்கூடிய கருவிகளாக பயன்படுத்துகிறது. ஒரு பிளகினில் பல `kernel_functions` குழுவாக இருக்கலாம்.

கீழே உள்ள உதாரணத்தில், இரண்டு செயல்பாடுகளைக் கொண்ட `DestinationsPlugin` உருவாக்குகிறோம்:  
1. `get_destinations` செயல்பாட்டைப் பயன்படுத்தி இடங்களின் பட்டியலை வழங்குகிறது  
2. ஒவ்வொரு இடத்திற்கும் கிடைக்கும் பட்டியலை `get_availabilty` செயல்பாட்டைப் பயன்படுத்தி வழங்குகிறது  


In [2]:
# Define a sample plugin for the sample
class DestinationsPlugin:
    """A List of Destinations for vacation."""

    @kernel_function(description="Provides a list of vacation destinations.")
    def get_destinations(self) -> Annotated[str, "Returns the vacation destinations."]:
        return """
        Barcelona, Spain
        Paris, France
        Berlin, Germany
        Tokyo, Japan
        New York, USA
        """

    @kernel_function(description="Provides the availability of a destination.")
    def get_availability(
        self, destination: Annotated[str, "The destination to check availability for."]
    ) -> Annotated[str, "Returns the availability of the destination."]:
        return """
        Barcelona - Unavailable
        Paris - Available
        Berlin - Available
        Tokyo - Unavailable
        New York - Available
        """

## கிளையன்ட் உருவாக்குதல்

இந்த மாதிரியில், [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ஐ பயன்படுத்தி LLM-க்கு அணுகுவோம்.

`ai_model_id` என்பது `gpt-4o-mini` என வரையறுக்கப்பட்டுள்ளது. GitHub Models சந்தையில் கிடைக்கும் மற்றொரு மாடலுக்கு மாறி, வெவ்வேறு முடிவுகளைப் பார்க்க முயற்சிக்கவும்.

GitHub Models-க்கு `base_url` பயன்படுத்தப்படும் `Azure Inference SDK` ஐ பயன்படுத்த, Semantic Kernel-இல் உள்ள `OpenAIChatCompletion` இணைப்பியை நாம் பயன்படுத்துவோம். Semantic Kernel-ஐ மற்ற மாடல் வழங்குநர்களுக்கு பயன்படுத்துவதற்கான [கிடைக்கக்கூடிய இணைப்பிகள்](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) கூட உள்ளன.


In [3]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ஏஜென்டை உருவாக்குதல்  
இப்போது நாம் ஏஜென்டின் பெயர் மற்றும் நாங்கள் அமைக்கக்கூடிய வழிமுறைகளைப் பயன்படுத்தி ஏஜென்டை உருவாக்குவோம்.  

நீங்கள் இந்த அமைப்புகளை மாற்றி, ஏஜென்டின் பதில்களில் உள்ள வேறுபாடுகளைப் பார்க்கலாம்.  


In [4]:
# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="TravelAgent",
    instructions="Answer questions about the travel destinations and their availability.",
    plugins=[DestinationsPlugin()],
)

## ஏஜென்டை இயக்குவது

இப்போது AI ஏஜென்டை இயக்குவோம். இந்த சுருக்கத்தில், `user_input`-இல் இரண்டு செய்திகளைச் சேர்த்து, ஏஜென்ட் தொடர்ச்சியான கேள்விகளுக்கு எப்படி பதிலளிக்கிறது என்பதை காணலாம்.

ஏஜென்ட் சரியான செயல்பாட்டை அழைத்து கிடைக்கக்கூடிய இடங்களின் பட்டியலைப் பெற வேண்டும் மற்றும் ஒரு குறிப்பிட்ட இடத்தின் கிடைப்பை உறுதிப்படுத்த வேண்டும்.

`user_inputs`-ஐ மாற்றி, ஏஜென்ட் எப்படி பதிலளிக்கிறது என்பதைப் பார்க்கலாம்.


In [6]:
user_inputs = [
    "What destinations are available?",
    "Is Barcelona available?",
    "Are there any vacation destinations available not in Europe?",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []
        function_calls_made = []  # Track function calls that were detected

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                        
                    # For the current semantic-kernel version, finalize function call immediately
                    # since FunctionResultContent may not be streamed
                    if current_function_name and argument_buffer:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls_made.append({
                            'name': current_function_name,
                            'args': formatted_args
                        })
                        
                elif isinstance(item, FunctionResultContent):
                    # This handles the case if FunctionResultContent is available (newer versions)
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # If we detected function calls but didn't get FunctionResultContent in streaming,
        # we can infer that functions were called successfully since we got a response
        if function_calls_made and not function_calls:
            for func_call in function_calls_made:
                function_calls.append(f"Calling function: {func_call['name']}({func_call['args']})")
            
            # Since we know functions were called and we have a response, 
            # we can indicate that function results were processed
            if len(function_calls_made) > 0:
                function_calls.append("\nFunction results were processed successfully (results used to generate the response above)")

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", AuthenticationError("Error code: 401 - {'error': {'code': 'unauthorized', 'message': 'Bad credentials', 'details': 'Bad credentials'}}"))


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
